In [ ]:
# The final step: LR testing with validation set is not working.
# Probably df and df_val have a different structure.
# instead of looking for the error I created a new file fify.ipynb 
# where I put everything up to the x-y split into a single function and ran it on both tables.
# check fifa.ipynb for the working version

# Imports

In [1]:
import pandas as pd
import numpy as np
import statistics as stats
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import seaborn as sns
from array import *
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler # do not use the function Normalise() - it does something entirely different
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error


# Basic Analysis & Cleaning Data

## Basic Analysis

In [2]:
df = pd.read_csv('fifa21_train.csv')
df_val = pd.read_csv('fifa21_validate.csv')

In [3]:
df.shape

(11701, 101)

In [4]:
df.head()

,ID,Name,Age,Nationality,Club,BP,Position,Team & Contract,Height,Weight,...,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK,OVA
0,184383,A. Pasche,26,Switzerland,FC Lausanne-Sport,CM,CM CDM,FC Lausanne-Sport 2015 ~ 2020,"5'9""",161lbs,...,59+1,59+1,59+1,58+1,54+1,54+1,54+1,58+1,15+1,64
1,188044,Alan Carvalho,30,China PR,Beijing Sinobo Guoan FC,ST,ST LW LM,"Beijing Sinobo Guoan FC Dec 31, 2020 On Loan","6'0""",159lbs,...,53+2,53+2,57+2,53+2,48+2,48+2,48+2,53+2,18+2,77
2,184431,S. Giovinco,33,Italy,Al Hilal,CAM,CAM CF,Al Hilal 2019 ~ 2022,"5'4""",134lbs,...,56+2,56+2,59+2,53+2,41+2,41+2,41+2,53+2,12+2,80
3,233796,J. Evans,22,Wales,Swansea City,CDM,CDM CM,Swansea City 2016 ~ 2021,"5'10""",152lbs,...,58+2,58+2,56+2,57+2,58+2,58+2,58+2,57+2,14+2,59
4,234799,Y. Demoncy,23,France,US Orléans Loiret Football,CDM,CDM CM,US Orléans Loiret Football 2018 ~ 2021,"5'11""",150lbs,...,64+2,64+2,64+2,63+2,61+2,61+2,61+2,63+2,15+2,65


In [5]:
# Standarize header
df.drop(['ID'], axis=1, inplace=True)
df.columns= df.columns.str.lower().str.replace(' ', '_')
df.columns= df.columns.str.lower().str.replace('/', '_')
df.columns= df.columns.str.lower().str.replace('&', 'and')

In [6]:
# Standarize header
df_val.drop(['ID'], axis=1, inplace=True)
df_val.columns= df.columns.str.lower().str.replace(' ', '_')
df_val.columns= df.columns.str.lower().str.replace('/', '_')
df_val.columns= df.columns.str.lower().str.replace('&', 'and')

In [7]:
df.columns

Index(['name', 'age', 'nationality', 'club', 'bp', 'position',
       'team_and_contract', 'height', 'weight', 'foot', 'growth', 'joined',
       'loan_date_end', 'value', 'wage', 'release_clause', 'contract',
       'attacking', 'crossing', 'finishing', 'heading_accuracy',
       'short_passing', 'volleys', 'skill', 'dribbling', 'curve',
       'fk_accuracy', 'long_passing', 'ball_control', 'movement',
       'acceleration', 'sprint_speed', 'agility', 'reactions', 'balance',
       'power', 'shot_power', 'jumping', 'stamina', 'strength', 'long_shots',
       'mentality', 'aggression', 'interceptions', 'positioning', 'vision',
       'penalties', 'composure', 'defending', 'marking', 'standing_tackle',
       'sliding_tackle', 'goalkeeping', 'gk_diving', 'gk_handling',
       'gk_kicking', 'gk_positioning', 'gk_reflexes', 'total_stats',
       'base_stats', 'w_f', 'sm', 'a_w', 'd_w', 'ir', 'pac', 'sho', 'pas',
       'dri', 'def', 'phy', 'hits', 'ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf',

In [8]:
# df.describe().T

## Dropping columns with no information

In [9]:
same = df['lf'] == df['rf']
same.value_counts()
same = df['lf'] == df['cf']  
same.value_counts()
df.drop(['lf', 'rf'], axis=1, inplace=True)
same = df['ls'] == df['st']  
same.value_counts()
same = df['ls'] == df['rs']  
same.value_counts()
df.drop(['ls', 'rs'], axis=1, inplace=True)
same = df['lam'] == df['ram']  
same.value_counts()
same = df['lam'] == df['cam']  
same.value_counts()
df.drop(['lam', 'ram'], axis=1, inplace=True)
same = df['lm'] == df['rm']  
same.value_counts()
df.drop(['rm'], axis=1, inplace=True)
same = df['lcm'] == df['rcm']  
same.value_counts()
same = df['cm'] == df['rcm']  
same.value_counts()
df.drop(['rcm', 'lcm'], axis=1, inplace=True)
same = df['ldm'] == df['rdm']  
same.value_counts()
same = df['ldm'] == df['cdm']  
same.value_counts()
df.drop(['rdm', 'ldm'], axis=1, inplace=True)
same = df['lwb'] == df['rwb']  
same.value_counts()
df.drop(['rwb'], axis=1, inplace=True)
same = df['lb'] == df['rb']  
same.value_counts()
df.drop(['rb'], axis=1, inplace=True)
same = df['lcb'] == df['rcb']  
same.value_counts()
same = df['lcb'] == df['cb']  
same.value_counts()
df.drop(['lcb', 'rcb'], axis=1, inplace=True)
same = df['cf'] == df['rw']  
same.value_counts()
same = df['lw'] == df['rw']  
same.value_counts()
df.drop(['rw'], axis=1, inplace=True)
df['test'] = df['club'] + ' ' + df['contract']
same = df['test'] == df['team_and_contract']
same.value_counts()
df.drop(['test'], axis=1, inplace=True)
df.drop(['team_and_contract'], axis=1, inplace=True)

In [10]:
same = df_val['lf'] == df_val['rf']
same.value_counts()
same = df_val['lf'] == df_val['cf']  
same.value_counts()
df_val.drop(['lf', 'rf'], axis=1, inplace=True)
same = df_val['ls'] == df_val['st']  
same.value_counts()
same = df_val['ls'] == df_val['rs']  
same.value_counts()
df_val.drop(['ls', 'rs'], axis=1, inplace=True)
same = df_val['lam'] == df_val['ram']  
same.value_counts()
same = df_val['lam'] == df_val['cam']  
same.value_counts()
df_val.drop(['lam', 'ram'], axis=1, inplace=True)
same = df_val['lm'] == df_val['rm']  
same.value_counts()
df_val.drop(['rm'], axis=1, inplace=True)
same = df_val['lcm'] == df_val['rcm']  
same.value_counts()
same = df_val['cm'] == df_val['rcm']  
same.value_counts()
df_val.drop(['rcm', 'lcm'], axis=1, inplace=True)
same = df_val['ldm'] == df_val['rdm']  
same.value_counts()
same = df_val['ldm'] == df_val['cdm']  
same.value_counts()
df_val.drop(['rdm', 'ldm'], axis=1, inplace=True)
same = df_val['lwb'] == df_val['rwb']  
same.value_counts()
df_val.drop(['rwb'], axis=1, inplace=True)
same = df_val['lb'] == df_val['rb']  
same.value_counts()
df_val.drop(['rb'], axis=1, inplace=True)
same = df_val['lcb'] == df_val['rcb']  
same.value_counts()
same = df_val['lcb'] == df_val['cb']  
same.value_counts()
df_val.drop(['lcb', 'rcb'], axis=1, inplace=True)
same = df_val['cf'] == df_val['rw']  
same.value_counts()
same = df_val['lw'] == df_val['rw']  
same.value_counts()
df_val.drop(['rw'], axis=1, inplace=True)
df_val['test'] = df_val['club'] + ' ' + df_val['contract']
same = df_val['test'] == df_val['team_and_contract']
same.value_counts()
df_val.drop(['test'], axis=1, inplace=True)
df_val.drop(['team_and_contract'], axis=1, inplace=True)

In [11]:
df.shape

(11701, 83)

## Check for duplicates

In [12]:
dfcopy = df.copy()
print(dfcopy.shape)
dfcopy = dfcopy.copy()
dfcopy = dfcopy.drop_duplicates()
print(dfcopy.shape)
dfcopy

(11701, 83)
(11701, 83)


,name,age,nationality,club,bp,position,height,weight,foot,growth,...,cf,cam,lm,cm,lwb,cdm,lb,cb,gk,ova
0,A. Pasche,26,Switzerland,FC Lausanne-Sport,CM,CM CDM,"5'9""",161lbs,Right,1,...,62+0,63+1,63+1,63+1,59+1,59+1,58+1,54+1,15+1,64
1,Alan Carvalho,30,China PR,Beijing Sinobo Guoan FC,ST,ST LW LM,"6'0""",159lbs,Right,0,...,77+0,76+1,76+1,68+2,57+2,53+2,53+2,48+2,18+2,77
2,S. Giovinco,33,Italy,Al Hilal,CAM,CAM CF,"5'4""",134lbs,Right,0,...,79+0,80+0,79+1,74+2,59+2,56+2,53+2,41+2,12+2,80
3,J. Evans,22,Wales,Swansea City,CDM,CDM CM,"5'10""",152lbs,Right,13,...,51+0,53+2,53+2,56+2,56+2,58+2,57+2,58+2,14+2,59
4,Y. Demoncy,23,France,US Orléans Loiret Football,CDM,CDM CM,"5'11""",150lbs,Right,8,...,59+0,61+2,62+2,63+2,64+2,64+2,63+2,61+2,15+2,65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11696,B. Böðvarsson,25,Iceland,Jagiellonia Białystok,LB,LB,"6'1""",168lbs,Left,5,...,52+0,54+2,58+2,56+2,63+2,60+2,63+2,61+2,16+2,65
11697,G. Gallon,27,France,ESTAC Troyes,GK,GK,"6'1""",174lbs,Right,4,...,26+0,24+2,25+2,25+2,25+2,26+2,24+2,26+2,69+2,70
11698,J. Flores,22,Chile,CD Antofagasta,RM,LM CAM RM,"5'6""",143lbs,Right,8,...,65+0,65+2,66+2,58+2,49+2,44+2,45+2,35+2,17+2,67
11699,Anderson Silva,26,Brazil,Barnsley,CM,NaN,"6'2""",179lbs,Right,7,...,68+0,72+0,71+0,71+0,66+0,68+0,64+0,60+0,25+0,68


## Check and deal with NaN values.

In [13]:
na_cols = df.columns[df.isna().any()]
df[na_cols].isna().sum()

club                 18
position            288
joined               41
loan_date_end     11072
volleys              41
curve                41
agility              41
balance              41
jumping              41
interceptions         7
positioning           7
vision               41
composure           279
sliding_tackle       41
a_w                  59
d_w                  59
dtype: int64

In [14]:
na_cols = df_val.columns[df_val.isna().any()]
df_val[na_cols].isna().sum()

club                 3
position            55
joined               3
loan_date_end     1889
volleys              3
curve                3
agility              3
balance              3
jumping              3
vision               3
composure           50
sliding_tackle       3
a_w                  8
d_w                  8
dtype: int64

In [15]:
#>90% of this column doesn't have a value
df.drop(['loan_date_end'], axis=1, inplace=True)

In [16]:
#>90% of this column doesn't have a value
df_val.drop(['loan_date_end'], axis=1, inplace=True)

In [17]:
na_cols = df.columns[df.isna().any()]

In [18]:
df[na_cols].dtypes

club               object
position           object
joined             object
volleys           float64
curve             float64
agility           float64
balance           float64
jumping           float64
interceptions     float64
positioning       float64
vision            float64
composure         float64
sliding_tackle    float64
a_w                object
d_w                object
dtype: object

In [19]:
summary = df.describe().T
summary['iqr'] = summary['75%']-summary['25%'] # adds Interquartile Range
summary['range'] = summary['max'] - summary['min'] # adds range# 

for col in summary.columns: # rounds all values in summary
    summary[col] = summary[col].apply(lambda x : round(x, 2))
summary

,count,mean,std,min,25%,50%,75%,max,iqr,range
age,11701.0,25.27,4.96,16.0,21.0,25.0,29.0,43.0,8.0,27.0
growth,11701.0,5.53,5.81,-1.0,0.0,4.0,10.0,26.0,10.0,27.0
attacking,11701.0,258.48,72.27,33.0,232.0,271.0,306.0,437.0,74.0,404.0
crossing,11701.0,51.59,17.87,6.0,41.0,56.0,65.0,94.0,24.0,88.0
finishing,11701.0,48.05,19.40,3.0,33.0,52.0,64.0,95.0,31.0,92.0
heading_accuracy,11701.0,53.49,16.96,5.0,46.0,56.0,65.0,93.0,19.0,88.0
short_passing,11701.0,60.44,13.97,8.0,56.0,63.0,69.0,94.0,13.0,86.0
volleys,11660.0,45.07,17.73,4.0,32.0,47.0,59.0,90.0,27.0,86.0
skill,11701.0,266.50,76.84,43.0,232.0,279.0,319.0,470.0,87.0,427.0
dribbling,11701.0,57.85,18.05,5.0,53.0,63.0,70.0,96.0,17.0,91.0


In [20]:
# Replace numerical values with the mean
# mean & median have been quite similar 
# round them, so they can be converted to int like the other values in this column
for col in na_cols:
    na_cols = df.columns[df.isna().any()]
    if df[col].dtypes == 'float64' or df[col].dtypes == 'int64':
        df[col].fillna(round(np.mean(df[col]),0), inplace=True)

In [21]:
# Replace null categorical values with 'unknown'
for col in na_cols:
    if df[col].dtypes == 'object':
        df[col].fillna('unknown', inplace=True)

In [22]:
# Replace numerical values with the mean
# mean & median have been quite similar 
# round them, so they can be converted to int like the other values in this column
for col in na_cols:
    na_cols = df_val.columns[df_val.isna().any()]
    if df_val[col].dtypes == 'float64' or df_val[col].dtypes == 'int64':
        df_val[col].fillna(round(np.mean(df_val[col]),0), inplace=True)
		
# Replace null categorical values with 'unknown'
for col in na_cols:
    if df_val[col].dtypes == 'object':
        df_val[col].fillna('unknown', inplace=True)

In [23]:
na_cols = df.columns[df.isna().any()]
df[na_cols].isna().sum()

Series([], dtype: float64)

In [24]:
df_val['composure'].fillna(round(np.mean(df_val['composure']),0), inplace=True)
df_val['volleys'].fillna(round(np.mean(df_val['volleys']),0), inplace=True)
df_val['curve'].fillna(round(np.mean(df_val['curve']),0), inplace=True)
df_val['agility'].fillna(round(np.mean(df_val['agility']),0), inplace=True)
df_val['balance'].fillna(round(np.mean(df_val['balance']),0), inplace=True)
df_val['jumping'].fillna(round(np.mean(df_val['jumping']),0), inplace=True)
df_val['vision'].fillna(round(np.mean(df_val['vision']),0), inplace=True)
df_val['sliding_tackle'].fillna(round(np.mean(df_val['sliding_tackle']),0), inplace=True)

In [25]:
na_cols = df_val.columns[df_val.isna().any()]
df_val[na_cols].isna().sum()

Series([], dtype: float64)

## convert to Datetime

In [26]:
df['joined'] = pd.to_datetime(df['joined'], errors='coerce')
df['joined'].fillna(stats.mode(df['joined']), inplace=True)


In [27]:
df_val['joined'] = pd.to_datetime(df_val['joined'], errors='coerce')
df_val['joined'].fillna(stats.mode(df_val['joined']), inplace=True)


## Cleaning content of columns (e.g. removing units)

### removing special characters

In [28]:
df['weight'] = df['weight'].str.replace('lbs','')
# df['value'] =  df['value'].str.replace('K','')
df['value'] =  df['value'].str.replace('€','')
df['w_f'] =  df['w_f'].str.replace('★','')
df['sm'] =  df['sm'].str.replace('★','')
df['ir'] =  df['ir'].str.replace('★','')
df['weight'] = df['weight'].astype(int)
df['w_f'] = df['w_f'].astype(int)
df['sm'] = df['sm'].astype(int)
df['ir'] = df['ir'].astype(int)


In [29]:
df_val['weight'] = df_val['weight'].str.replace('lbs','')
# df_val['value'] =  df_val['value'].str.replace('K','')
df_val['value'] =  df_val['value'].str.replace('€','')
df_val['w_f'] =  df_val['w_f'].str.replace('★','')
df_val['sm'] =  df_val['sm'].str.replace('★','')
df_val['ir'] =  df_val['ir'].str.replace('★','')
df_val['weight'] = df_val['weight'].astype(int)
df_val['w_f'] = df_val['w_f'].astype(int)
df_val['sm'] = df_val['sm'].astype(int)
df_val['ir'] = df_val['ir'].astype(int)


### inch to cm 

In [30]:
heightlist = []
for row in df['height']:
    row = row.replace('"', '')
    split = row.split("'")
    height = int(split[0]) * 30.48 + int(split[1]) * 2.54
    heightlist.append(height)
df['height'] = heightlist       
df['height'] = df['height'].astype(int)

In [31]:
heightlist = []
for row in df_val['height']:
    row = row.replace('"', '')
    split = row.split("'")
    height = int(split[0]) * 30.48 + int(split[1]) * 2.54
    heightlist.append(height)
df_val['height'] = heightlist       
df_val['height'] = df_val['height'].astype(int)

### remove the + and create a new column for the value behind the +

In [32]:
for col in ['st', 'lw','cf', 'cam', 'lm', 'cm', 'lwb', 'cdm', 'lb', 'cb', 'gk']:
    colvalue = []
    coladdvalue = []
    for row in df[col]:
        split = row.split("+")
        colvalue.append(split[0])
        coladdvalue.append(split[1])
    df[col] = colvalue
    df[col + 'add'] = coladdvalue
    df[col] = df[col].astype(int)
    df[col + 'add'] = df[col + 'add'].astype(int)     
    df['st'] = df['st'].astype(int)   
    

In [33]:
for col in ['st', 'lw','cf', 'cam', 'lm', 'cm', 'lwb', 'cdm', 'lb', 'cb', 'gk']:
    colvalue = []
    coladdvalue = []
    for row in df_val[col]:
        split = row.split("+")
        colvalue.append(split[0])
        coladdvalue.append(split[1])
    df_val[col] = colvalue
    df_val[col + 'add'] = coladdvalue
    df_val[col] = df_val[col].astype(int)
    df_val[col + 'add'] = df_val[col + 'add'].astype(int)     
    df_val['st'] = df_val['st'].astype(int)   
    

### add a column for the date when the contract ends

In [34]:
contract_to = []
for row in df['contract']:
    split = row.split(" ~ ")  
    if len(split) == 2:
        contract_to.append(split[1])   
    else:
        contract_to.append('0')
df['contract_to'] = contract_to       
df['contract_to'] = df['contract_to'].astype(int)   
# df['contract_to'] = pd.to_datetime(df['contract_to'], errors='coerce')

In [35]:
contract_to = []
for row in df_val['contract']:
    split = row.split(" ~ ")  
    if len(split) == 2:
        contract_to.append(split[1])   
    else:
        contract_to.append('0')
df_val['contract_to'] = contract_to       
df_val['contract_to'] = df_val['contract_to'].astype(int)   
# df_val['contract_to'] = pd.to_datetime(df_val['contract_to'], errors='coerce')

### Cleaning Currency-Columns

In [36]:
#correcting values
df['hits_K'] = df['hits'].astype(str).str[-1]
df['hits_K'] = df['hits_K'].str.replace('0','1')
df['hits_K'] = df['hits_K'].str.replace('K','100')
df['hits_K'] = pd.to_numeric(df['hits_K'])
df['hits'] = df['hits'].str.replace('K','')
df['hits'] = pd.to_numeric(df['hits'])
df['hits'] = round(df['hits'] * df['hits_K'],2)
df.drop(['hits_K'], axis=1, inplace=True)

In [37]:
#correcting values
df['value_KoM'] = df['value'].astype(str).str[-1]
df['value_KoM'] = df['value_KoM'].str.replace('0','1')
df['value_KoM'] = df['value_KoM'].str.replace('M','100000')
df['value_KoM'] = df['value_KoM'].str.replace('K','100')
df['value_KoM'] = pd.to_numeric(df['value_KoM'])
df['value'] = df['value'].str.replace('€','')
df['value'] = df['value'].str.replace('M','')
df['value'] = df['value'].str.replace('K','')
df['value'] = pd.to_numeric(df['value'])
df['value'] = round(df['value'] * df['value_KoM'],2)
df.drop(['value_KoM'], axis=1, inplace=True)

In [38]:
#correcting wages
df['wage_KoM'] = df['wage'].astype(str).str[-1]
df['wage_KoM'] = df['wage_KoM'].str.replace('0','1')
df['wage_KoM'] = df['wage_KoM'].str.replace('M','1000000')
df['wage_KoM'] = df['wage_KoM'].str.replace('K','1000')
df['wage_KoM'] = pd.to_numeric(df['wage_KoM'])
df['wage'] = df['wage'].str.replace('€','')
df['wage'] = df['wage'].str.replace('M','')
df['wage'] = df['wage'].str.replace('K','')
df['wage'] = pd.to_numeric(df['wage'])
df['wage'] = round(df['wage'] * df['wage_KoM'],2)
df.drop(['wage_KoM'], axis=1, inplace=True)

In [39]:
#correcting release_clauses
df['release_clause_KoM'] = df['release_clause'].astype(str).str[-1]
df['release_clause_KoM'] = df['release_clause_KoM'].str.replace('0','1')
df['release_clause_KoM'] = df['release_clause_KoM'].str.replace('M','100000')
df['release_clause_KoM'] = df['release_clause_KoM'].str.replace('K','100')
df['release_clause_KoM'] = pd.to_numeric(df['release_clause_KoM'])
df['release_clause'] = df['release_clause'].str.replace('€','')
df['release_clause'] = df['release_clause'].str.replace('M','')
df['release_clause'] = df['release_clause'].str.replace('K','')
df['release_clause'] = pd.to_numeric(df['release_clause'])
df['release_clause'] = round(df['release_clause'] * df['release_clause_KoM'],2)
df.drop(['release_clause_KoM'], axis=1, inplace=True)

In [40]:
#correcting values
df_val['hits_K'] = df_val['hits'].astype(str).str[-1]
df_val['hits_K'] = df_val['hits_K'].str.replace('0','1')
df_val['hits_K'] = df_val['hits_K'].str.replace('K','100')
df_val['hits_K'] = pd.to_numeric(df_val['hits_K'])
df_val['hits'] = df_val['hits'].str.replace('K','')
df_val['hits'] = pd.to_numeric(df_val['hits'])
df_val['hits'] = round(df_val['hits'] * df_val['hits_K'],2)
df_val.drop(['hits_K'], axis=1, inplace=True)
#correcting values
df_val['value_KoM'] = df_val['value'].astype(str).str[-1]
df_val['value_KoM'] = df_val['value_KoM'].str.replace('0','1')
df_val['value_KoM'] = df_val['value_KoM'].str.replace('M','100000')
df_val['value_KoM'] = df_val['value_KoM'].str.replace('K','100')
df_val['value_KoM'] = pd.to_numeric(df_val['value_KoM'])
df_val['value'] = df_val['value'].str.replace('€','')
df_val['value'] = df_val['value'].str.replace('M','')
df_val['value'] = df_val['value'].str.replace('K','')
df_val['value'] = pd.to_numeric(df_val['value'])
df_val['value'] = round(df_val['value'] * df_val['value_KoM'],2)
df_val.drop(['value_KoM'], axis=1, inplace=True)
#correcting wages
df_val['wage_KoM'] = df_val['wage'].astype(str).str[-1]
df_val['wage_KoM'] = df_val['wage_KoM'].str.replace('0','1')
df_val['wage_KoM'] = df_val['wage_KoM'].str.replace('M','1000000')
df_val['wage_KoM'] = df_val['wage_KoM'].str.replace('K','1000')
df_val['wage_KoM'] = pd.to_numeric(df_val['wage_KoM'])
df_val['wage'] = df_val['wage'].str.replace('€','')
df_val['wage'] = df_val['wage'].str.replace('M','')
df_val['wage'] = df_val['wage'].str.replace('K','')
df_val['wage'] = pd.to_numeric(df_val['wage'])
df_val['wage'] = round(df_val['wage'] * df_val['wage_KoM'],2)
df_val.drop(['wage_KoM'], axis=1, inplace=True)
#correcting release_clauses
df_val['release_clause_KoM'] = df_val['release_clause'].astype(str).str[-1]
df_val['release_clause_KoM'] = df_val['release_clause_KoM'].str.replace('0','1')
df_val['release_clause_KoM'] = df_val['release_clause_KoM'].str.replace('M','100000')
df_val['release_clause_KoM'] = df_val['release_clause_KoM'].str.replace('K','100')
df_val['release_clause_KoM'] = pd.to_numeric(df_val['release_clause_KoM'])
df_val['release_clause'] = df_val['release_clause'].str.replace('€','')
df_val['release_clause'] = df_val['release_clause'].str.replace('M','')
df_val['release_clause'] = df_val['release_clause'].str.replace('K','')
df_val['release_clause'] = pd.to_numeric(df_val['release_clause'])
df_val['release_clause'] = round(df_val['release_clause'] * df_val['release_clause_KoM'],2)
df_val.drop(['release_clause_KoM'], axis=1, inplace=True)

## Rearranging the columns

In [41]:
# df.columns

In [42]:
df = df[['name', 'age', 'nationality', 'club', 'bp', 'position', 'height',
       'weight', 'foot', 'growth', 'joined', 'value', 'wage', 'release_clause',
       'contract', 'contract_to', 'attacking', 'crossing', 'finishing', 'heading_accuracy',
       'short_passing', 'volleys', 'skill', 'dribbling', 'curve',
       'fk_accuracy', 'long_passing', 'ball_control', 'movement',
       'acceleration', 'sprint_speed', 'agility', 'reactions', 'balance',
       'power', 'shot_power', 'jumping', 'stamina', 'strength', 'long_shots',
       'mentality', 'aggression', 'interceptions', 'positioning', 'vision',
       'penalties', 'composure', 'defending', 'marking', 'standing_tackle',
       'sliding_tackle', 'goalkeeping', 'gk_diving', 'gk_handling',
       'gk_kicking', 'gk_positioning', 'gk_reflexes', 'total_stats',
       'base_stats', 'w_f', 'sm', 'a_w', 'd_w', 'ir', 'pac', 'sho', 'pas',
       'dri', 'def', 'phy', 'hits', 'st','stadd',  'lw','lwadd', 'cf','cfadd', 'cam','camadd', 'lm', 'lmadd', 'cm', 'lwb','lwbadd',
       'cmadd','cdm','cdmadd', 'lb', 'lbadd','cb','cbadd', 'gk', 'gkadd', 'ova' 
       ]]

In [43]:
df_val = df_val[['name', 'age', 'nationality', 'club', 'bp', 'position', 'height',
       'weight', 'foot', 'growth', 'joined', 'value', 'wage', 'release_clause',
       'contract', 'contract_to', 'attacking', 'crossing', 'finishing', 'heading_accuracy',
       'short_passing', 'volleys', 'skill', 'dribbling', 'curve',
       'fk_accuracy', 'long_passing', 'ball_control', 'movement',
       'acceleration', 'sprint_speed', 'agility', 'reactions', 'balance',
       'power', 'shot_power', 'jumping', 'stamina', 'strength', 'long_shots',
       'mentality', 'aggression', 'interceptions', 'positioning', 'vision',
       'penalties', 'composure', 'defending', 'marking', 'standing_tackle',
       'sliding_tackle', 'goalkeeping', 'gk_diving', 'gk_handling',
       'gk_kicking', 'gk_positioning', 'gk_reflexes', 'total_stats',
       'base_stats', 'w_f', 'sm', 'a_w', 'd_w', 'ir', 'pac', 'sho', 'pas',
       'dri', 'def', 'phy', 'hits', 'st','stadd',  'lw','lwadd', 'cf','cfadd', 'cam','camadd', 'lm', 'lmadd', 'cm', 'lwb','lwbadd',
       'cmadd','cdm','cdmadd', 'lb', 'lbadd','cb','cbadd', 'gk', 'gkadd', 'ova' 
       ]]

In [44]:
df_val.dtypes

name           object
age             int64
nationality    object
club           object
bp             object
                ...  
cb              int32
cbadd           int32
gk              int32
gkadd           int32
ova             int64
Length: 94, dtype: object

# Deeper analysis

## X_y split & Normalizing the data & One Hot Encoder


In [45]:
df.shape

(11701, 94)

In [46]:
df_val.shape

(1999, 94)

In [47]:
# df = pd.concat([df, df_val], axis=0)

In [48]:
df.shape

(11701, 94)

In [49]:
y = df['value']
X = df.drop(['value'], axis=1)
X_num = X._get_numeric_data()
X_cat = X.select_dtypes('object')

In [50]:
# Normalizing data: make data range from 0 - 1, instead of from min to max
transformer = MinMaxScaler().fit(X_num)
x_normalized = transformer.transform(X_num)
print(x_normalized.shape)
x_normalized #this is an array
X_num = pd.DataFrame(x_normalized, columns=X_num.columns) #now we convert the array to a dataframe

(11701, 83)


In [51]:
X_cat = X_cat.drop(['name'], axis=1)
X_cat = X_cat.drop(['position'], axis=1)
X_cat = X_cat.drop(['contract'], axis=1)
X_cat = X_cat.drop(['nationality'], axis=1)
X_cat = X_cat.drop(['club'], axis=1)

In [52]:
X_cat

,bp,foot,a_w,d_w
0,CM,Right,High,Medium
1,ST,Right,High,Low
2,CAM,Right,High,Medium
3,CDM,Right,Medium,Medium
4,CDM,Right,Low,Medium
...,...,...,...,...
11696,LB,Left,Medium,Medium
11697,GK,Right,Medium,Medium
11698,RM,Right,High,Medium
11699,CM,Right,unknown,unknown


In [53]:
encoder = OneHotEncoder().fit(X_cat)
encoded_for_p = encoder.transform(X_cat).toarray()
encoded_for_p
encoder.categories_

cols=[colname for row in encoder.categories_ for colname in row]
cols
onehot_encoded_for_p = pd.DataFrame(encoded_for_p,columns=cols)
cols_to_drop=[row[0] for row in encoder.categories_]
cols_to_drop
onehot_encoded_for_p = onehot_encoded_for_p.drop(cols_to_drop,axis=1)
onehot_encoded_for_p.head()
# print(encoded_for_p)
# print(cols)


,CB,CDM,CF,CM,GK,LB,LM,LW,LWB,RB,...,RW,RWB,ST,Right,Low,Medium,unknown,Low,Medium,unknown
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0


In [54]:
X = pd.concat([X_num, onehot_encoded_for_p], axis=1)
# X = X_num


In [55]:
na_cols = X.columns[X.isna().any()]
X[na_cols] = X[na_cols].dropna
X[na_cols].isna().sum()

Series([], dtype: float64)

In [56]:
y

0         52500.0
1        850000.0
2        900000.0
3         27500.0
4         72500.0
           ...   
11696     62500.0
11697    160000.0
11698    110000.0
11699         0.0
11700     70000.0
Name: value, Length: 11701, dtype: float64

In [57]:
na_cols = X.columns[X.isna().any()]
X[na_cols].isna().sum()

Series([], dtype: float64)

# Linear Regression

## Basic

In [58]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=35)


In [59]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)



(9360, 104)
(2341, 104)
(9360,)
(2341,)


In [60]:
lm = linear_model.LinearRegression()
lm.fit(X_train,y_train)

LinearRegression()

## Linear Regression testing with validation setb

In [65]:
# The next step is not working. Probably df and df_val have a different structure.
# instead of looking for the error I put everything up to the x-y split into a single function and ran it on both tables.
# check fifa.ipynb for the working version

In [61]:
y_val = df_val['value']
X_val = df_val.drop(['value'], axis=1)
X_val_num = X_val._get_numeric_data()
X_val_cat = X_val.select_dtypes('object')


X_val_cat = X_val_cat.drop(['name'], axis=1)
X_val_cat = X_val_cat.drop(['position'], axis=1)
X_val_cat = X_val_cat.drop(['contract'], axis=1)
X_val_cat = X_val_cat.drop(['nationality'], axis=1)
X_val_cat = X_val_cat.drop(['club'], axis=1)

# Normalizing data: make data range from 0 - 1, instead of from min to max
transformer = MinMaxScaler().fit(X_val_num)
x_normalized = transformer.transform(X_val_num)
print(x_normalized.shape)
x_normalized #this is an array
X_val_num = pd.DataFrame(x_normalized, columns=X_val_num.columns) #now we convert the array to a dataframe

# onehotencoding
encoder = OneHotEncoder().fit(X_val_cat)
encoded_for_p = encoder.transform(X_val_cat).toarray()
encoded_for_p
encoder.categories_
cols=[colname for row in encoder.categories_ for colname in row]
cols
onehot_encoded_for_p = pd.DataFrame(encoded_for_p,columns=cols)
cols_to_drop=[row[0] for row in encoder.categories_]
cols_to_drop
onehot_encoded_for_p = onehot_encoded_for_p.drop(cols_to_drop,axis=1)
onehot_encoded_for_p.head()


X_val = pd.concat([X_val_num, onehot_encoded_for_p], axis=1)
# X = X_num



(1999, 83)


In [62]:
predictions = lm.predict(X_test)

r2_score(y_test, predictions)



-1.4425673778503215e+19

In [63]:
## Use a different ratio of train test split

In [64]:
print('Highest R2Score for testsize = ' + str(highest_i))
print('Max(score) = ' + str(max(scores))) 

NameError: name 'highest_i' is not defined

## Calculating average of r2 score

## Show Heatmap

%%time
correlations_matrix = round(numerical.corr(),2)
sns.heatmap(correlations_matrix, annot=True)
plt.show()

In [ ]:
# df_onehotencoded = pd.concat([X, y,], axis=1)
# correlations_matrix = round(df_onehotencoded.corr(),2)
# plt.subplots(figsize=(55,40))
# sns.heatmap(correlations_matrix, annot=True)
# plt.show()

%%time
correlations_matrix = round(numerical.corr(),2)
plt.subplots(figsize=(55,40))
sns.heatmap(correlations_matrix, annot=True)
plt.show()